# 图像清洁小工具
fastai提供了几个小工具来支持深度学习从业者的工作流程。 小部件的目的是帮助您组织，清理和准备模型的数据。 窗口小部件按数据类型分隔。

In [9]:
from fastai.vision import *
from fastai.basics import *
from fastai.callbacks import *
from fastai.widgets import *

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [3]:
learn = cnn_learner(data, models.resnet18, metrics=error_rate)

In [4]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,error_rate,time
0,0.186349,0.102232,0.035329,01:21
1,0.099124,0.081136,0.025515,01:16


In [5]:
learn.save('stage-1')

我们使用训练集中的所有数据创建一个databunch，并且没有验证集（DatasetFormatter仅使用训练集）

In [6]:
db = (ImageList.from_folder(path)
                   .split_none()
                   .label_from_folder()
                   .databunch())

In [7]:
learn = cnn_learner(db, models.resnet18, metrics=[accuracy])
learn.load('stage-1');

---

## class DatasetFormatter

`DatasetFormatter()`

返回具有要显示的相应格式和文件索引的数据集。

DatasetFormatter类通过返回基于指定的DatasetType的格式化DatasetTfm来为小部件准备图像数据集。 使用from_toplosses直接从学习者那里获取最有问题的图像。 （可选）您可以将返回的格式化数据集限制为n_imgs。

**from_similars**

`from_similars(learn, layer_ls:list=[0, 7, 2], **kwargs)`

获取最相似图像的索引。

**from_toplosses**

`from_toplosses(learn, n_imgs=None, **kwargs)`

获得最高损失的指数。

---

## class ImageCleaner

`ImageCleaner(dataset, fns_idxs, path, batch_size:int=5, duplicates=False)`

显示用于重新标记或删除的图像，并将路径中的更改保存为“cleaning.csv”。

ImageCleaner用于清理不属于数据集的图像。 它将图像连续呈现，并为您提供从文件系统中删除文件的机会。 要使用ImageCleaner，我们必须首先使用DatasetFormatter（）。from_toplosses来获取错误分类图像的建议索引。

In [10]:
ds, idxs = DatasetFormatter().from_toplosses(learn)

In [11]:
ImageCleaner(ds, idxs, path)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

ImageCleaner不会更改磁盘上的任何内容（标签或图像的存在）。 相反，它会在您的数据路径中创建一个'cleaning.csv'文件，您需要从该文件中为文件加载新的数据清单以应用更改。

In [15]:
df = pd.read_csv(path/'cleaned.csv', header='infer')
df

,name,label
0,train\3\10.png,3
1,train\3\10000.png,3
2,train\3\10011.png,3
3,train\3\10031.png,3
4,train\3\10034.png,3
5,train\3\10042.png,3
6,train\3\10052.png,3
7,train\3\1007.png,3
8,train\3\10074.png,3
9,train\3\10091.png,3


In [16]:
# We create a databunch from our csv. We include the data in the training set and we don't use a validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
db = (ImageList.from_df(df, path)
                   .split_none()
                   .label_from_df()
                   .databunch(bs=64))

In [17]:
learn = cnn_learner(db, models.resnet18, metrics=error_rate)
learn = learn.load('stage-1')

然后，您可以再次使用ImageCleaner在数据集中查找重复项。 为此，您可以在从.from_similars获取索引和数据集后调用ImageCleaner时指定duplicates = True。 请注意，如果您正在使用具有维度（n_batches，n_features，1,1）的图层输出，那么您不需要任何池（最后一层就是这种情况）。 建议使用.from_similars（）和resnet使用最后一层而不使用池，就像在下面的单元格中一样。

In [18]:
ds, idxs = DatasetFormatter().from_similars(learn, layer_ls=[0,7,1], pool=None)

Getting activations...


Computing similarities...


In [19]:
ImageCleaner(ds, idxs, path, duplicates=True)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

In [ ]:
doc(ImageDownloader)

---

## class ImageDownloader

`ImageDownloader(path:PathOrStr='data')`

显示一个小部件，允许在Jupyter笔记本或实验室中搜索和下载谷歌图像搜索中的图像。

ImageDownloader小部件为您提供了一种快速引导图像数据集而无需离开笔记本的方法。 它搜索并下载符合搜索条件和分辨率/质量要求的图像，并将它们存储在提供的路径中的文件系统上。

每个搜索查询（或标签）的图像都存储在路径中的单独文件夹中。 例如，如果你用设置为./data的路径瞳孔老虎，你将获得一个文件夹./data/tiger/，里面有老虎图像。

ImageDownloader将在下载后使用verify_images（）自动清理并验证下载的图像。

In [24]:
path = Config.data_path()/'image_downloader'
os.makedirs(path, exist_ok=True)
ImageDownloader(path)

在Jupyter笔记本之外的python脚本中下载图像

In [25]:
path = Config.data_path()/'image_downloader'
files = download_google_images(path, 'aussie shepherd', size='>1024*768', n_images=30)

len(files)

30

**download_google_images**

`download_google_images(path:PathOrStr, search_term:str, size:str=`'>400300'`*, n_images:int=10, format:str='jpg', max_workers:int=16, timeout:int=4) → FilePathList`

在Google上搜索n_images图像，匹配search_term和大小要求，将它们下载到path / search_term并使用max_workers线程验证它们。

使用ImageDownloader填充图像后，您可以通过调用ImageDataBunch.from_folder（path，size = size）或使用数据块API来获取ImageDataBunch。

In [26]:
# Setup path and labels to search for
path = Config.data_path()/'image_downloader'
labels = ['boston terrier', 'french bulldog']

# Download images
for label in labels: 
    download_google_images(path, label, size='>400*300', n_images=50)

# Build a databunch and train! 
src = (ImageList.from_folder(path)
       .split_by_rand_pct()
       .label_from_folder()
       .transform(get_transforms(), size=224))

db  = src.databunch(bs=16, num_workers=0)

learn = cnn_learner(db, models.resnet34, metrics=[accuracy])
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,1.866978,0.790729,0.833333,00:08
1,1.123303,0.482663,0.866667,00:07
2,0.788141,0.492640,0.866667,00:07


**下载超过一百张图片**

要获取超过一百张图像，ImageDownloader使用selenium和chromedriver滚动浏览Google图像搜索结果页面并刮取图像网址。 默认情况下，它们不是必需的依赖项。 如果您的系统上没有安装它们，则窗口小部件将显示错误消息。

要安装硒，只需在您的fastai环境中安装硒。

在Mac上，您可以使用brew cask install chromedriver安装chromedriver。

在Ubuntu上看看最新的Chromedriver版本，然后是：

```
wget https://chromedriver.storage.googleapis.com/2.45/chromedriver_linux64.zip
unzip chromedriver_linux64.zip
```

请注意，在100张图像下下载不需要除fastai本身之外的任何依赖，但是下载超过一百张图像使用selenium和chromedriver。

大小可以是以下之一：

```
'>400*300'
'>640*480'
'>800*600'
'>1024*768'
'>2MP'
'>4MP'
'>6MP'
'>8MP'
'>10MP'
'>12MP'
'>15MP'
'>20MP'
'>40MP'
'>70MP'
```